<a href="https://colab.research.google.com/github/RusAl84/text_mining/blob/master/%D0%94%D0%973%2002112021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TF-IDF

У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

tfidf=tf∗idf

idf=log((N+1)/(Nw+1))+1

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: TfidfVectorizer, рассмотрим его.

In [1]:
texts = ": \u041A\u0430\u043A \u043F\u0435\u0440\u0435\u0434\u0430\u043B\u043E \u0430\u0433\u0435\u043D\u0442\u0441\u0442\u0432\u043E \"\u0418\u043D\u0442\u0435\u0440\u0444\u0430\u043A\u0441\", \u0441\u0435\u0433\u043E\u0434\u043D\u044F \u0432 \u0447\u0430\u0441 \u0434\u043D\u044F \u043F\u043E \u043C\u043E\u0441\u043A\u043E\u0432\u0441\u043A\u043E\u043C\u0443 \u0432\u0440\u0435\u043C\u0435\u043D\u0438 \u0432\u043E\u0437\u043B\u0435 \u0437\u0430\u0431\u043E\u0440\u0430 \u0435\u0432\u0440\u0435\u0439\u0441\u043A\u043E\u0439 \u0448\u043A\u043E\u043B\u044B, \u043D\u0430\u0445\u043E\u0434\u044F\u0449\u0435\u0439\u0441\u044F \u043D\u0430 \u041B\u0435\u043D\u0438\u043D\u0441\u043A\u043E\u043C \u043F\u0440\u043E\u0441\u043F\u0435\u043A\u0442\u0435 (\u042E\u0433\u043E-\u0417\u0430\u043F\u0430\u0434\u043D\u044B\u0439 \u0430\u0434\u043C\u0438\u043D\u0438\u0441\u0442\u0438\u0432\u043D\u044B\u0439 \u043E\u043A\u0440\u0443\u0433) \u0432 \u041C\u043E\u0441\u043A\u0432\u0435 \u043E\u0431\u043D\u0430\u0440\u0443\u0436\u0435\u043D\u0430 \u043C\u0438\u043D\u0430. \u0418\u0437 \u041C\u0438\u043D\u0438\u0441\u0442\u0435\u0440\u0441\u0442\u0432\u0430 \u043F\u043E \u0447\u0440\u0435\u0437\u0432\u044B\u0447\u0430\u0439\u043D\u044B\u043C \u0441\u0438\u0442\u0443\u0430\u0446\u0438\u044F\u043C \u0420\u0424 \u0441\u043E\u043E\u0431\u0449\u0430\u044E\u0442, \u0447\u0442\u043E \u043C\u0435\u0441\u0442\u043E\u043D\u0430\u0445\u043E\u0436\u0434\u0435\u043D\u0438\u0435 \u0432\u0437\u0440\u044B\u0432\u043E\u043E\u043F\u0430\u0441\u043D\u043E\u0433\u043E \u043F\u0440\u0435\u0434\u043C\u0435\u0442\u0430 \u043E\u0446\u0435\u043F\u043B\u0435\u043D\u043E \u0441\u043E\u0442\u0440\u0443\u0434\u043D\u0438\u043A\u0430\u043C\u0438 \u043C\u0438\u043B\u0438\u0446\u0438\u0438. \u041A \u043C\u0435\u0441\u0442\u0443 \u043F\u0440\u043E\u0438\u0441\u0448\u0435\u0441\u0442\u0432\u0438\u044F \u0441\u043F\u0435\u0448\u043D\u043E \u043F\u0440\u0438\u0431\u044B\u043B\u0438 \u0441\u043F\u0435\u0446\u0438\u0430\u043B\u0438\u0441\u0442\u044B \u043C\u043E\u0441\u043A\u043E\u0432\u0441\u043A\u043E\u0433\u043E \u043F\u043E\u0438\u0441\u043A\u043E\u0432\u043E-\u0441\u043F\u0430\u0441\u0430\u0442\u0435\u043B\u044C\u043D\u043E\u0433\u043E \u043E\u0442\u0440\u044F\u0434\u0430 \u041F\u0421\u041E-6 \u0438 \u043F\u0438\u0440\u043E\u0442\u0435\u0445\u043D\u0438\u043A\u0438 \u041C\u0438\u043D\u0438\u0441\u0442\u0435\u0440\u0441\u0442\u0432\u0430 \u043E\u0431\u043E\u0440\u043E\u043D\u044B. \u0422\u0438\u043F \u0438 \u043F\u0440\u043E\u0438\u0441\u0445\u043E\u0436\u0434\u0435\u043D\u0438\u0435 \u043C\u0438\u043D\u044B \u0443\u0441\u0442\u0430\u043D\u043E\u0432\u043B\u0435\u043D\u044B: \u044D\u0442\u043E \u043C\u0438\u043D\u043E\u043C\u0435\u0442\u043D\u0430\u044F \u043C\u0438\u043D\u0430 \u0441 \u043E\u043F\u0435\u0440\u0435\u043D\u0438\u0435\u043C \u0432\u0440\u0435\u043C\u0435\u043D \u0412\u0435\u043B\u0438\u043A\u043E\u0439 \u041E\u0442\u0435\u0447\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u043E\u0439 \u0432\u043E\u0439\u043D\u044B. \u0415\u0435 \u043D\u0430\u0448\u043B\u0438 \u043F\u0440\u0438 \u0437\u0435\u043C\u043B\u044F\u043D\u044B\u0445 \u0440\u0430\u0431\u043E\u0442\u0430\u0445, \u0432\u044B\u043F\u043E\u043B\u043D\u044F\u0432\u0448\u0438\u0445\u0441\u044F \u0440\u044F\u0434\u043E\u043C \u0441\u043E \u0448\u043A\u043E\u043B\u043E\u0439. \u0417\u0430 \u043F\u043E\u0441\u043B\u0435\u0434\u043D\u0435\u0435 \u0434\u0432\u043E\u0435 \u0441\u0443\u0442\u043E\u043A \u044D\u0442\u043E \u0443\u0436\u0435 \u0442\u0440\u0435\u0442\u044C\u0435 \u0427\u041F \u0441\u043E \u0432\u0437\u0440\u044B\u0432\u0447\u0430\u0442\u044B\u043C\u0438 \u0432\u0435\u0449\u0435\u0441\u0442\u0432\u0430\u043C\u0438 \u0432 \u041C\u043E\u0441\u043A\u0432\u0435: \u043F\u043E\u0437\u0430\u0432\u0447\u0435\u0440\u0430 \u043F\u0440\u043E\u0433\u0440\u0435\u043C\u0435\u043B \u0432\u0437\u0440\u044B\u0432 \u0432 \u0442\u043E\u0440\u0433\u043E\u0432\u043E\u043C \u043A\u043E\u043C\u043F\u043B\u0435\u043A\u0441\u0435 \"\u041E\u0445\u043E\u0442\u043D\u044B\u0439 \u0440\u044F\u0434\" \u043D\u0430 \u041C\u0430\u043D\u0435\u0436\u043D\u043E\u0439 \u043F\u043B\u043E\u0449\u0430\u0434\u0438, \u0441\u0435\u0433\u043E\u0434\u043D\u044F \u0431\u044B\u043B \u043D\u0430\u0439\u0434\u0435\u0442 \u043F\u043E\u0434\u043E\u0437\u0440\u0438\u0442\u0435\u043B\u044C\u043D\u044B\u0439 \u043F\u0430\u043A\u0435\u0442 \u043D\u0430 \u041F\u0430\u0432\u0435\u043B\u0435\u0446\u043A\u043E\u043C \u0432\u043E\u043A\u0437\u0430\u043B\u0435." #@param {type:"string"}
stexts = texts
print(texts)

: Как передало агентство "Интерфакс", сегодня в час дня по московскому времени возле забора еврейской школы, находящейся на Ленинском проспекте (Юго-Западный администивный округ) в Москве обнаружена мина. Из Министерства по чрезвычайным ситуациям РФ сообщают, что местонахождение взрывоопасного предмета оцеплено сотрудниками милиции. К месту происшествия спешно прибыли специалисты московского поисково-спасательного отряда ПСО-6 и пиротехники Министерства обороны. Тип и происхождение мины установлены: это минометная мина с оперением времен Великой Отечественной войны. Ее нашли при земляных работах, выполнявшихся рядом со школой. За последнее двое суток это уже третье ЧП со взрывчатыми веществами в Москве: позавчера прогремел взрыв в торговом комплексе "Охотный ряд" на Манежной площади, сегодня был найдет подозрительный пакет на Павелецком вокзале.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [3]:
def printTF_IDF(texts):
  records_count = 30
  tfIdfTransformer = TfidfVectorizer(ngram_range=(1, 4), use_idf=True, max_features=records_count)
  countVectorizer = CountVectorizer(ngram_range=(1, 4), max_features=records_count)
  wordCount = countVectorizer.fit_transform([texts])
  TfIdf = tfIdfTransformer.fit_transform([texts])
  names = countVectorizer.get_feature_names()
  df=[]
  df = pd.DataFrame(list(names), columns=['names'])
  df = df.assign(Word_Count=wordCount.T.todense())
  df = df.assign(TF_IDF=TfIdf.T.todense())
  df = df.sort_values('TF_IDF', ascending=False)
  print(df)

In [4]:
printTF_IDF(texts)

                                       names  Word_Count    TF_IDF
4                                         на           3  0.390567
29                                       это           2  0.260378
2                               министерства           2  0.260378
3                                     москве           2  0.260378
5                                         по           2  0.260378
28                                        со           2  0.260378
27                                   сегодня           2  0.260378
1                                       мина           2  0.260378
22                             происхождение           1  0.130189
20                  прогремел взрыв торговом           1  0.130189
21        прогремел взрыв торговом комплексе           1  0.130189
26                                       ряд           1  0.130189
23                        происхождение мины           1  0.130189
24            происхождение мины установлены           1  0.13

удалим знаки переноса строк и сделаем текст в нижнем регистре

In [5]:
  str2 = ''
  for item in texts.split():
    str2 = str2 + ' ' + item
  texts = str(texts)
  texts = texts.lower()
  # print(texts)
  texts = texts.replace('\n', ' ')

удалим цифры

In [6]:
  str2 = ''
  for c in texts:
      if c not in ('0', "1", '2', '3', '4', '5', '6', '7', '8', '9', '«', '»', '–', "\""):
          str2 = str2 + c
  texts = str2
  str2 = ''

удалим знаки пунктуации

In [7]:
import string
pattern = string.punctuation
for c in texts:
    if c not in pattern:
        str2 = str2 + c
    else:
        str2 = str2 + " "
texts = str2
str2 = ''

удалим стоп слова

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
  from nltk.corpus import stopwords
  russian_stopwords = stopwords.words("russian")
  for word in texts.split():
      if word not in (russian_stopwords):
          str2 = str2 + " " + word
  texts = str2

  str2 = ''
  for word in texts.split():
      if len(word) > 1:
          str2 = str2 + " " + word
  texts = str2
  print(texts)

 передало агентство интерфакс сегодня час дня московскому времени возле забора еврейской школы находящейся ленинском проспекте юго западный администивный округ москве обнаружена мина министерства чрезвычайным ситуациям рф сообщают местонахождение взрывоопасного предмета оцеплено сотрудниками милиции месту происшествия спешно прибыли специалисты московского поисково спасательного отряда псо пиротехники министерства обороны тип происхождение мины установлены это минометная мина оперением времен великой отечественной войны нашли земляных работах выполнявшихся рядом школой последнее двое суток это третье чп взрывчатыми веществами москве позавчера прогремел взрыв торговом комплексе охотный ряд манежной площади сегодня найдет подозрительный пакет павелецком вокзале


рассчитаем TF-IDF

In [10]:
printTF_IDF(texts)

                                         names  Word_Count    TF_IDF
29                                         это           2  0.298142
2                                 министерства           2  0.298142
3                                       москве           2  0.298142
28                                     сегодня           2  0.298142
1                                         мина           2  0.298142
17              происхождение мины установлены           1  0.149071
27                                         ряд           1  0.149071
26  рф сообщают местонахождение взрывоопасного           1  0.149071
25                 рф сообщают местонахождение           1  0.149071
24                                 рф сообщают           1  0.149071
23                                          рф           1  0.149071
22          работах выполнявшихся рядом школой           1  0.149071
21     происшествия спешно прибыли специалисты           1  0.149071
20                         происше

#Суммаризации текстов 

Задача суммаризации текстов (автореферирование) - одна из ключевых, широко обсуждаемых задач NLP. Она состоит в сжатии больших объемов текста до связного краткого содержания, отражающего только основные идеи.


#Алгоритм TextRank
TextRank - это алгоритм, основанный на PageRank, который часто используется для извлечения ключевых слов и суммирования текста. 
PageRank (PR) - это алгоритм, используемый для расчета веса веб-страниц. 


In [11]:
from gensim.summarization import summarize, keywords
print("Исходный текст:")
print(stexts)
print("Результат работы TextRank:")
print(summarize(str(stexts)))

Исходный текст:
: Как передало агентство "Интерфакс", сегодня в час дня по московскому времени возле забора еврейской школы, находящейся на Ленинском проспекте (Юго-Западный администивный округ) в Москве обнаружена мина. Из Министерства по чрезвычайным ситуациям РФ сообщают, что местонахождение взрывоопасного предмета оцеплено сотрудниками милиции. К месту происшествия спешно прибыли специалисты московского поисково-спасательного отряда ПСО-6 и пиротехники Министерства обороны. Тип и происхождение мины установлены: это минометная мина с оперением времен Великой Отечественной войны. Ее нашли при земляных работах, выполнявшихся рядом со школой. За последнее двое суток это уже третье ЧП со взрывчатыми веществами в Москве: позавчера прогремел взрыв в торговом комплексе "Охотный ряд" на Манежной площади, сегодня был найдет подозрительный пакет на Павелецком вокзале.
Результат работы TextRank:
: Как передало агентство "Интерфакс", сегодня в час дня по московскому времени возле забора еврейск

#Алгоритм Rake
RAKE: Rapid Automatic Keyword Extraction Algorithm
Алгоритм RAKE извлекает ключевые слова с помощью основанного на разделителе подхода, чтобы идентифицировать ключевые слова кандидата и баллы их использующий совместные встречаемости слова, которые появляются в ключевых словах кандидата. Ключевые слова могут содержать несколько лексем. Кроме того, алгоритм RAKE также объединяет ключевые слова, когда они кажутся многократно, разделенными тем же разделителем слияния.


In [12]:
!pip install rake_nltk
from rake_nltk import Metric, Rake
r = Rake(language="russian")
import nltk
nltk.download('punkt')
r.extract_keywords_from_text(stexts)
mas = r.get_ranked_phrases()
set2 = set()
for item in mas:
    if not "nan" in str(item).replace(" nan ", " "):
        set2.add(str(item).replace(" nan ", " "))
mas = list(set2)
print("Исходный текст:")
print(stexts)
print("Результат работы TextRammk:")
print(str(mas))

     |████████████████████████████████| 1.5 MB 3.6 MB/s 
     |████████████████████████████████| 748 kB 37.2 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Исходный текст:
: Как передало агентство "Интерфакс", сегодня в час дня по московскому времени возле забора еврейской школы, находящейся на Ленинском проспекте (Юго-Западный администивный округ) в Москве обнаружена мина. Из Министерства по чрезвычайным ситуациям РФ сообщают, что местонахождение взрывоопасного предмета оцеплено сотрудниками милиции. К месту происшествия спешно прибыли специалисты московского поисково-спасательного отряда ПСО-6 и пиротехники Министерства обороны. Тип и происхождение мины установлены: это минометная мина с оперением времен Великой Отечественной войны. Ее нашли при земляных работах, выполнявшихся рядом со школой. За последнее двое суток это уже третье ЧП со взрывчатыми веществами в Москве: позавчера прогремел взрыв в торговом комплексе "Охотный ряд" на Манежной площади, сегодня был найдет подозрительный пакет на Павелецком вокзале.
Результат работы Tex

#Тематическое моделирование
Тематическое моделирование — это метод извлечения тем из текста. Latent Dirichlet Allocation (LDA) — популярный алгоритм моделирования тем реализованные в том числе в пакете Gensim. Основная задача алгоритмов ТМ, заключается в том что бы полученные темы были хорошего качество, понятными, самозначимыми и разделенными. Достижение этих целей во многом зависит от качества предварительной обработки текста и стратегии поиска оптимального количества тем. 

##Импорт пакетов


In [13]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Подход LDA к тематическому моделированию заключается в том, что каждый документ рассматривается как набор тем в определенной пропорции. И каждая тема как набор ключевых слов, опять же, в определенной пропорции.

После того, как вы укажете алгоритму количество тем, все, что он сделает, — это отобразит распределение тем в документах и распределение ключевых слов по темам.

Тема — это не что иное, как набор доминирующих ключевых слов. Просто взглянув на ключевые слова, вы сможете определить, о чем эта тема.

Ниже приведены ключевые факторы для получения хороших разделительных тем:

Качество обработки текста.
Разнообразие тем, о которых говорится в тексте.
Выбор алгоритма моделирование тем.
Количество тем, указанных в алгоритме.
Алгоритмы настройки параметров.

# Подготовим стоп-слова


In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

##Лемматизация
Лемматизация — это не что иное, как преобразование слова в его корневое слово. Например: лемма слова «machines» — это «machine». Аналогично, «walking» -> «walk», «mice» -> «mouse» и так далее.

In [15]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
ltexts = ""
print(texts)
for word in texts.split():
    if len(str(word)) > 2:
        ltexts+=" " + morph.parse(word)[0].normal_form
print(ltexts)

     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 8.2 MB 7.0 MB/s 
 передало агентство интерфакс сегодня час дня московскому времени возле забора еврейской школы находящейся ленинском проспекте юго западный администивный округ москве обнаружена мина министерства чрезвычайным ситуациям рф сообщают местонахождение взрывоопасного предмета оцеплено сотрудниками милиции месту происшествия спешно прибыли специалисты московского поисково спасательного отряда псо пиротехники министерства обороны тип происхождение мины установлены это минометная мина оперением времен великой отечественной войны нашли земляных работах выполнявшихся рядом школой последнее двое суток это третье чп взрывчатыми веществами москве позавчера прогремел взрыв торговом комплексе охотный ряд манежной площади сегодня найдет подозрительный пакет павелецком вокзале
 передать агентство интерфакс сегодня час день московский время возле забор еврейский школа находиться ленинский прос

##Создадим словарь и корпус.
Двумя основными входными данными для тематической модели LDA являются словарь (id2word) и корпус. Давайте создадим их.

In [16]:
from nltk.util import ngrams
def make_bigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    bigrams = list(ngrams(token, 2))
    # print(bigrams)
    return bigrams
def make_trigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    trigrams = list(ngrams(token, 2))
    return trigrams

btexts = make_bigrams(texts) + make_trigrams(texts)
id2word = corpora.Dictionary(btexts)
corpus = [id2word.doc2bow(text) for text in btexts]
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('агентство', 1), ('передало', 1)]]

Или вы можете увидеть удобочитаемую форму самого корпуса.

In [17]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('агентство', 1), ('передало', 1)]]

##Построим тематическую модель
У нас есть все необходимое для обучения модели LDA. В дополнение к корпусу и словарю необходимо также указать количество тем.

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

##Просмотр тем в модели LDA
Вышеупомянутая модель LDA построена на 20 различных темах, где каждая тема представляет собой комбинацию ключевых слов, и каждое ключевое слово вносит определенный вес в тему.

Вы можете увидеть ключевые слова для каждой темы и вес (важность) каждого ключевого слова, используя lda_model.print_topics().

In [19]:
lda_model.print_topics()

[(0,
  '0.049*"спасательного" + 0.049*"обнаружена" + 0.049*"взрывчатыми" + 0.049*"пакет" + 0.049*"проспекте" + 0.049*"веществами" + 0.049*"москве" + 0.042*"ленинском" + 0.026*"комплексе" + 0.026*"чп"'),
 (1,
  '0.051*"псо" + 0.051*"западный" + 0.051*"манежной" + 0.051*"ряд" + 0.051*"сообщают" + 0.051*"работах" + 0.051*"выполнявшихся" + 0.051*"рядом" + 0.027*"охотный" + 0.027*"рф"'),
 (2,
  '0.099*"сегодня" + 0.060*"третье" + 0.060*"министерства" + 0.042*"час" + 0.042*"интерфакс" + 0.042*"дня" + 0.042*"забора" + 0.042*"еврейской" + 0.032*"пиротехники" + 0.032*"рф"'),
 (3,
  '0.051*"министерства" + 0.051*"чрезвычайным" + 0.051*"мины" + 0.051*"сотрудниками" + 0.051*"двое" + 0.051*"предмета" + 0.051*"суток" + 0.051*"спешно" + 0.051*"последнее" + 0.051*"оцеплено"'),
 (4,
  '0.046*"мина" + 0.046*"это" + 0.046*"москве" + 0.046*"московского" + 0.046*"времен" + 0.046*"прогремел" + 0.046*"позавчера" + 0.046*"оперением" + 0.046*"округ" + 0.046*"отечественной"')]

##Визуализация темы и ключевых слов
Теперь, когда модель LDA создана, следующим шагом является изучение созданных тем и связанных с ними ключевых слов. Нет лучшего инструмента, чем интерактивная диаграмма пакета pyLDAvis.

In [20]:
!pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

     |████████████████████████████████| 1.6 MB 3.3 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=7cd5fd7e9dd3660023fa173bfa2485fb0bc569642d96a01ae3661954d5c21cb1
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [21]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.187314 -0.117734       1        1  21.804955
0     -0.070838 -0.026818       2        1  20.680295
3     -0.041139  0.219123       3        1  19.631685
1      0.171964 -0.121950       4        1  19.627685
2      0.127327  0.047378       5        1  18.255380, topic_info=             Term      Freq     Total Category  logprob  loglift
3         сегодня  6.000000  6.000000  Default  30.0000  30.0000
22   министерства  7.000000  7.000000  Default  29.0000  29.0000
65         третье  4.000000  4.000000  Default  28.0000  28.0000
19         москве  7.000000  7.000000  Default  27.0000  27.0000
23   чрезвычайным  4.000000  4.000000  Default  26.0000  26.0000
..            ...       ...       ...      ...      ...      ...
26       сообщают  0.174015  4.120694   Topic5  -5.9001  -1.4639
53        великой  0.174093  4.162858   Topic5  -5.8997  -1.4737
54  отечественной  0.174093  4.162859   Topic5  -5.8997  -1.4737
21           мина  0.174016  7.507564   Topic5  -5.9001  -2.0638
19         москве  0.174015  7.540476   Topic5  -5.9001  -2.0682

[196 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         4  0.315209      агентство
0         5  0.315209      агентство
17        1  0.482885  администивный
17        4  0.482885  администивный
53        1  0.960878        великой
...     ...       ...            ...
49        1  0.523099            это
49        3  0.261550            это
49        5  0.261550            это
15        2  0.481468            юго
15        4  0.481468            юго

[124 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 2, 3])